In [48]:
# automatically update imports
%load_ext autoreload
%autoreload 2

# For graphs
%matplotlib inline

# snippet for relative imports
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [63]:
from analysis import Analysis
#city_ref = 'grenoble_france' 
city_ref = 'avila_spain'

city = Analysis(city_ref, pois_shp_path = "cities/"+city_ref+"/full_uses.shp", grid_step=.005)

In [64]:
city.pois.head()

Querying locally for `['pois']`
`['pois']` is/are not stored locally. Determining it/them through `get_extracted_osm_points` method
('Downloaded and unzipped:', 'https://s3.amazonaws.com/metro-extracts.mapzen.com/avila_spain.osm2pgsql-shapefiles.zip')
('Clip polygon file:', 'cities/avila_spain_osm_polygon.shp')
('Clip point file:', 'cities/avila_spain_osm_point.shp')
('Processing cut number:', 0)
quadrants/_pts_q0
('extract_point_uses. Loaded Points:', 9)
('Empty shape for file:', 'cities/avila_spain/residential_pts_q0')
('Empty shape for file:', 'cities/avila_spain/activities_pts_q0')
('Activites points', 0)
('Residential points', 0)
('extract_poly_uses. Loaded Points:', 1)
('Empty shape for file:', 'cities/avila_spain/residential_poly_q0')
('Empty shape for file:', 'cities/avila_spain/activities_poly_q0')
('Activites polygons', 0)
('Residential polygons', 0)
('infer_poly_uses. Polygons with Landuse Region:', 7, 'Polygons to infer', 0)
--- 0.155574083328 seconds ---
('Small buildings 

,category,key,lat,lon,value
id,,,,,
325315918,activity,leisure,40.629154,-4.648201,pitch
325315920,activity,leisure,40.629008,-4.647757,pitch
325315921,activity,leisure,40.628869,-4.647456,pitch
3544687641,activity,shop,40.641578,-4.690898,butcher
3545585660,activity,shop,40.646708,-4.684789,butcher


In [ ]:
city.kde

In [ ]:
xx, yy = city.grid

import plots
figsize = (12,6)
plots.plot_(xx,yy,city.f_kde_act,'KDE: Activity', figsize)
plots.plot_(xx,yy,city.f_kde_res,'KDE: Residential', figsize)

In [ ]:
###
import lu_mix
lu_mix_metrics = lu_mix.get_phi_metrics()
lu_mix.normalisation_max = False # True

for i in lu_mix_metrics: # Plot all metrics
    city._f_lu_mix_grid = None
    city._phi_metric = i
    plots.plot_contour(xx,yy,city.f_lu_mix_grid,"Land use mix: "+i)
    plots.plot_(xx,yy,city.f_lu_mix_grid,'Land use mixity: '+i, figsize)

In [ ]:
city.lu_mix

In [ ]:
city.moran

In [ ]:
city.relative_entropy

In [ ]:
%matplotlib inline
city.scatter_pois(overlap=True)